In [147]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... 
  - https://repo.continuum.io/pkgs/main/win-64/win-64::_tflow_1100_select-0.0.3-mkl, https://repo.continuum.io/pkgs/main/win-64/win-64::conda-env-2.6.0-1, https://repo.continuum.io/pkgs/main/win-64/win-64::intel-openmp-2018.0.3-0, https://repo.continuum.io/pkgs/main/win-64/win-64::vs2013_runtime-12.0.21005-1
  - defaults/win-64::_tflow_1100_select-0.0.3-mkl, https://repo.continuum.io/pkgs/main/win-64/win-64::conda-env-2.6.0-1, https://repo.continuum.io/pkgs/main/win-64/win-64::intel-openmp-2018.0.3-0, https://repo.continuum.io/pkgs/main/win-64/win-64::vs2013_runtime-12.0.21005-1
  - defaults/win-64::conda-env-2.6.0-1, https://repo.continuum.io/pkgs/main/win-64/win-64::_tflow_1100_select-0.0.3-mkl, https://repo.continuum.io/pkgs/main/win-64/win-64::intel-openmp-2018.0.3-0, https://repo.continuum.io/pkgs/main/win-64/win-64::vs2013_runtime-12.0.21005-1
  - defaults/win-64::_tflow_1100_select-0.0.3-mkl, defaults/win-64::conda-env-2.6.0-1, https://repo.co

<h1>
Assignment 2: Segmenting and Clustering Neighborhoods in Toronto
</h1>

<h4>
    <center> Data Science Capstone </center>
</h4>

In [148]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

#for web-scrapping
import requests
import lxml.html as lh

print('Libraries imported.')

Libraries imported.


In [149]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

<h4>
    Reading url using Pandas easily
</h4>

In [150]:
lol = pd.read_html(url)
col=['PostalCode', 'Borough', 'Neighborhood']
lol = pd.DataFrame(lol[0])
lol = lol[~lol.Borough.str.contains('Not assigned')]
lol.reset_index(drop=True, inplace=True)
lol.drop_duplicates()
lol['Neighborhood'].replace("",lol['Borough'], inplace=True)
lol

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [151]:
#data=pd.DataFrame()
data=[]
col=['PostalCode', 'Borough', 'Neighborhood']
for t in tr_elements[1:]:
    name=[]
    for T in t:
        element=T.text_content().rstrip()
        name.append(element)
    data.append(name)

In [152]:
init_df = pd.DataFrame(data[:-4], columns=col)
init_df = init_df[~init_df.Borough.str.contains('Not assigned')]
init_df.reset_index(drop=True, inplace=True)
init_df.drop_duplicates()
init_df['Neighborhood'].replace("",init_df['Borough'], inplace=True)

In [153]:
init_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [154]:
init_df.shape

(103, 3)

<h3>
Part 2: Latitude and Longitude columns in Dataframe
</h3>

For each neighborhood we need to find latitude and longitude, so we will iterate through each row adding columns Latitude and Longitude

In [155]:
latitude=[]
longitude=[]
for i in range(len(init_df)):
    address = init_df['Neighborhood'][i].split(',')[0]
    geolocator = Nominatim(user_agent="tor_explorer")
    location = geolocator.geocode(address)
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    print (i ,longitude[i], latitude[i])

0 -122.22068778004532 37.8567738
1 -79.3111892 43.732658
2 -79.3604569 43.6607056
3 -79.4375067 43.7220788
4 -79.3903399 43.659659
5 -79.5888776 43.7869052
6 -2.3258985 52.1159559
7 -79.3459439 43.775347
8 -106.3983612 41.0219162
9 -79.3771141 43.6564995
10 -79.4406853 43.7087117
11 -79.5685684 43.6631995
12 -79.1304992 43.7802711
13 -79.3459439 43.775347
14 -79.3191316 43.6999302
15 -79.3727041 43.6694032
16 -79.4249313241846 43.690844350000006
17 -79.5765162 43.6622732
18 -79.1982293 43.7552251
19 -79.296712 43.6710244
20 -79.37539591138858 43.64798435
21 -72.01977897507511 44.42006025
22 -71.1522766 42.4792618
23 -79.3680904 43.7047983
24 -74.208741 40.8081888
25 -79.4184051 43.6641106
26 -114.1293235 50.9563182
27 -79.3650189 43.7996637
28 149.5804943 -33.4166378
29 -79.3454074 43.704553
30 -77.43428 37.5385087
31 -80.1691941 44.0497496
32 -79.2116324 43.7437422
33 -90.1637318 40.6333724
34 -79.50448 43.7541351
35 -79.3839347 43.6534817
36 103.8224032 1.2653951
37 -79.4311163254604

AttributeError: 'NoneType' object has no attribute 'latitude'